In [1]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 981 kB 8.1 MB/s 
     |████████████████████████████████| 358 kB 46.1 MB/s 
     |████████████████████████████████| 139 kB 56.0 MB/s 
     |████████████████████████████████| 55 kB 4.5 MB/s 
     |████████████████████████████████| 4.1 MB 46.6 MB/s 
     |████████████████████████████████| 58 kB 5.8 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.11 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
Get:1 https://

In [42]:
# 라이브러리 import
import os
import pandas as pd
import numpy as np
import math 

# 셀레늄
from selenium.webdriver.common.alert import Alert
from selenium import webdriver  # 라이브러리(모듈) 가져오라
# pip install chromedriver_autoinstaller
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains as AC
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

# tqdm : for문 진행상황 체크
from tqdm import tqdm, tqdm_notebook
from tqdm.notebook import tqdm

# 정규표현식(regular expression) : 문자(알파벳,한글), 숫자, 특수기호 정제 및 추출
import re
from time import sleep
import time

# 워닝 무시
import warnings
warnings.filterwarnings('ignore')

In [43]:
# Colab에서는 내부창을 띄울 수 없음
# 그래서 창이 안뜨도록 아래의 코드를 실행

options = webdriver.ChromeOptions()
options.add_argument('--headless')        # Head-less 설정
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver', options=options)

In [44]:
# 저장위치 설정을 위한 마운트

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
# test = [1,2,3,4,5]
# test1 = [10,20,30,40,50]
# test2 = [100,200,300,400,500]
# p = 2

# page_coll = {}

# page_coll['artist_name'] = test
# page_coll['song_name'] = test1
# page_coll['lyric'] = test2

# target_df = pd.DataFrame.from_dict(page_coll, orient='index')
# df = target_df.transpose()

# df.to_csv(f"/content/drive/MyDrive/00.이어드림/NLP/NLP_Project/윤상현/melon_ballad({p}).csv", encoding='utf-8-sig')

In [ ]:
# 페이지 할당 (301 - 400)


# 306 Page부터 시작
start_page = 307
list_url = f'https://www.melon.com/genre/song_list.htm?gnrCode=GN0100#params%5BgnrCode%5D=GN0100&params%5BdtlGnrCode%5D=&params%5BorderBy%5D=POP&params%5BsteadyYn%5D=N&po=pageObj&startIndex={(start_page*50)-49}'

driver.get(list_url)

time.sleep(2)


# 시작 페이지와 동일하게 시작 범위 설정 해주기
for p in tqdm(range(307,401)):

    page_num = p
    print("현재 페이지 : ",page_num)  

    artist_names = []
    song_names = []
    lyrics = []

    # 50개의 슬롯에 대한 가수, 곡명, 가사 저장
    for i in tqdm(range(1,51)):

        a = np.random.randint(3, 10, size=6)
        
        ## 가사 클릭 후 들어가기
        time.sleep(a[0])
        driver.find_element(By.CSS_SELECTOR,f"#frm > div > table > tbody > tr:nth-child({i}) > td:nth-child(4) > div > a").click()

        ## 파일명 설정 및 파일 생성

        # 가수 이름 추출
        time.sleep(a[1])
        artist_name = driver.find_element(By.CSS_SELECTOR, ".artist").text
        artist_names.append(artist_name)

        # 곡명 추출
        time.sleep(a[2])
        song_name = driver.find_element(By.CSS_SELECTOR, "div.song_name").text
        song_names.append(song_name)


        # 가사 있을 때

        time.sleep(a[3])
        try:
            # 펼치기 아이콘 클릭
            driver.find_element(By.CSS_SELECTOR, "button.button_more.arrow_d").click()

            time.sleep(a[4])
            # 가사 크롤링
            lyric = driver.find_element(By.ID, "d_video_summary").text
            lyrics.append(lyric)

            
        # 가사 없을 때
        except:
            lyric = "[가사 준비중]"
            lyrics.append(lyric)

        time.sleep(a[5])

        driver.back()


    a = np.random.randint(5, 10)
    time.sleep(a)

    page_coll = {}

    page_coll['artist_name'] = artist_names
    page_coll['song_name'] = song_names
    page_coll['lyric'] = lyrics

    target_df = pd.DataFrame.from_dict(page_coll, orient='index')
    df = target_df.transpose()

    # 경로는 각자의 경로에 맞게 설정
    df.to_csv(f"/content/drive/MyDrive/00.이어드림/NLP/NLP_Project/crawling data/Raw_data/melon_ballad({p}).csv", encoding='utf-8-sig')

    # url 설정 (해당 페이지에 대한 url)
    page_url = f'https://www.melon.com/genre/song_list.htm?gnrCode=GN0100#params%5BgnrCode%5D=GN0100&params%5BdtlGnrCode%5D=&params%5BorderBy%5D=POP&params%5BsteadyYn%5D=N&po=pageObj&startIndex={1+(50*p)}'
   
    
    # url 접속
    driver.get(page_url)

    a = np.random.randint(60,100)
    time.sleep(a)


  0%|          | 0/94 [00:00<?, ?it/s]

현재 페이지 :  307


  0%|          | 0/50 [00:00<?, ?it/s]

현재 페이지 :  308


  0%|          | 0/50 [00:00<?, ?it/s]

...

  0%|          | 0/50 [00:00<?, ?it/s]

현재 페이지 :  326


  0%|          | 0/50 [00:00<?, ?it/s]

현재 페이지 :  327


  0%|          | 0/50 [00:00<?, ?it/s]